<a href="https://colab.research.google.com/github/zzuupp/Object_Detection-Segmentation/blob/main/pytorch-objdet/Pytorch_Object_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Object Detection 구현

In [1]:
import numpy as np
import pandas as pd
import os
import sys

In [2]:
import torch
import torchvision
from torchvision import datasets, models
from torchvision.transforms import functional as FT
from torchvision import transforms as T
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader, sampler, random_split, Dataset
import copy
import math
from PIL import Image
import cv2
import albumentations as A
from pycocotools.coco import COCO
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
from collections import defaultdict, deque
import datetime
import time
from tqdm import tqdm
from torchvision.utils import draw_bounding_boxes
from torch.utils.tensorboard import SummaryWriter

In [4]:
Aqua_data_train_url  = '/content/drive/MyDrive/Aquarium Combined.v2-raw-1024.coco/train'
Aqua_data_valid_url = '/content/drive/MyDrive/Aquarium Combined.v2-raw-1024.coco/valid'
Aqua_data_test_url = '/content/drive/MyDrive/Aquarium Combined.v2-raw-1024.coco/test'

In [ ]:
def get_transform(train = False):
    if train :
        transform = A.Compose([
            A.Resize(600, 600), # our input size can be 600px
            A.HorizontalFlip(p = 0.3), # 30% 확률로 좌우 뒤집기.
            A.VerticalFlip(p = 0.3),   # 30% 확률로 위아래 뒤집기.
            A.RandomBrightnessContrast(p = 0.1), # 10% 확률로 밝기/대비 무작위 변화.
            A.ColorJitter(p = 0.1), # 10% 확률로 색상/채도/명도 변화
            ToTensorV2() # NumPy => PyTorch Tensor 변환
        ], bbox_params = A.BboxParams(format = 'coco'))  #객체 탐지는 바운딩박스도 함께 변환해야함.